In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

# TODO: Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'cs231n/assignments/assignment1/'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# This downloads the CIFAR-10 dataset to your Drive
# if it doesn't already exist.
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/
!bash get_datasets.sh
%cd /content/drive/My\ Drive/$FOLDERNAME

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/cs231n/assignments/assignment1/cs231n/datasets
/content/drive/My Drive/cs231n/assignments/assignment1


# Softmax exercise

*Complete and hand in this completed worksheet (including its outputs and any supporting code outside of the worksheet) with your assignment submission. For more details see the [assignments page](http://vision.stanford.edu/teaching/cs231n/assignments.html) on the course website.*

This exercise is analogous to the SVM exercise. You will:

- implement a fully-vectorized **loss function** for the Softmax classifier
- implement the fully-vectorized expression for its **analytic gradient**
- **check your implementation** with numerical gradient
- use a validation set to **tune the learning rate and regularization** strength
- **optimize** the loss function with **SGD**
- **visualize** the final learned weights


In [ ]:
import random
import numpy as np
from cs231n.data_utils import load_CIFAR10
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

In [ ]:
def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000, num_dev=500):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the linear classifier. These are the same steps as we used for the
    SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
    
    # Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
    
    # subsample the data
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]
    mask = np.random.choice(num_training, num_dev, replace=False)
    X_dev = X_train[mask]
    y_dev = y_train[mask]
    
    # Preprocessing: reshape the image data into rows
    X_train = np.reshape(X_train, (X_train.shape[0], -1))
    X_val = np.reshape(X_val, (X_val.shape[0], -1))
    X_test = np.reshape(X_test, (X_test.shape[0], -1))
    X_dev = np.reshape(X_dev, (X_dev.shape[0], -1))
    
    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis = 0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image
    X_dev -= mean_image
    
    # add bias dimension and transform into columns
    X_train = np.hstack([X_train, np.ones((X_train.shape[0], 1))])
    X_val = np.hstack([X_val, np.ones((X_val.shape[0], 1))])
    X_test = np.hstack([X_test, np.ones((X_test.shape[0], 1))])
    X_dev = np.hstack([X_dev, np.ones((X_dev.shape[0], 1))])
    
    return X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test, X_dev, y_dev = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
print('dev data shape: ', X_dev.shape)
print('dev labels shape: ', y_dev.shape)

Train data shape:  (49000, 3073)
Train labels shape:  (49000,)
Validation data shape:  (1000, 3073)
Validation labels shape:  (1000,)
Test data shape:  (1000, 3073)
Test labels shape:  (1000,)
dev data shape:  (500, 3073)
dev labels shape:  (500,)


## Softmax Classifier

Your code for this section will all be written inside `cs231n/classifiers/softmax.py`.


In [ ]:
# First implement the naive softmax loss function with nested loops.
# Open the file cs231n/classifiers/softmax.py and implement the
# softmax_loss_naive function.

from cs231n.classifiers.softmax import softmax_loss_naive
import time

# Generate a random softmax weight matrix and use it to compute the loss.
W = np.random.randn(3073, 10) * 0.0001
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As a rough sanity check, our loss should be something close to -log(0.1).
print('loss: %f' % loss)
print('sanity check: %f' % (-np.log(0.1)))

loss: 2.385968
sanity check: 2.302585


In [ ]:
W = np.random.randn(3073, 10) * 0.0001
loss=0
scores=np.exp(X_dev.dot(W))
#soft_max_scores=scores/sum(scores,axis=1)
#soft_max_scores
#softmax=scores()
soft_max_score=np.divide(scores,np.reshape(np.sum(scores,axis=1),(-1,1)))
loss+=sum(-np.log(soft_max_score)[np.arange(500),y_dev[np.arange(500)]])
soft_max_score.shape
loss/500
y_dev

array([3, 0, 2, 3, 0, 1, 1, 2, 5, 6, 1, 5, 2, 0, 4, 4, 2, 9, 2, 2, 1, 9,
       2, 7, 1, 3, 8, 9, 0, 0, 2, 3, 1, 6, 1, 6, 3, 2, 3, 8, 5, 5, 5, 7,
       3, 2, 0, 5, 2, 0, 9, 2, 3, 9, 9, 0, 2, 8, 3, 2, 4, 0, 7, 1, 1, 8,
       9, 4, 1, 2, 9, 8, 6, 9, 3, 1, 7, 4, 8, 2, 3, 6, 5, 1, 0, 8, 9, 1,
       3, 9, 8, 7, 8, 6, 1, 1, 5, 6, 7, 9, 1, 6, 4, 1, 8, 8, 5, 2, 9, 2,
       4, 9, 7, 0, 8, 6, 0, 3, 4, 5, 6, 9, 4, 8, 4, 0, 5, 9, 9, 6, 1, 5,
       0, 1, 0, 8, 5, 8, 1, 5, 2, 7, 2, 3, 5, 2, 9, 0, 4, 2, 4, 7, 8, 0,
       8, 2, 7, 3, 8, 2, 8, 1, 3, 5, 7, 5, 4, 0, 7, 2, 3, 2, 4, 6, 6, 6,
       7, 2, 2, 3, 2, 5, 6, 1, 1, 4, 9, 4, 4, 5, 3, 9, 1, 6, 5, 4, 8, 3,
       5, 0, 9, 6, 5, 5, 0, 5, 2, 3, 2, 7, 6, 7, 9, 9, 5, 6, 6, 6, 0, 0,
       7, 1, 9, 0, 8, 1, 5, 8, 7, 0, 0, 3, 6, 0, 4, 9, 5, 7, 9, 6, 6, 1,
       8, 7, 6, 4, 0, 0, 4, 6, 6, 3, 1, 4, 4, 6, 8, 0, 7, 7, 4, 9, 9, 0,
       8, 9, 7, 4, 0, 1, 8, 3, 7, 0, 9, 9, 9, 5, 4, 5, 6, 5, 0, 4, 0, 8,
       3, 3, 6, 0, 0, 7, 0, 6, 3, 2, 7, 9, 0, 7, 3,

**Inline Question 1**

Why do we expect our loss to be close to -log(0.1)? Explain briefly.**

$\color{blue}{\textit Your Answer:}$ *훈련이 끝나면 모델이 잘 고르기 때문에 -log1에 근접하나 초기에는 선택 능력이 없으므로 전부다 같은 score를 주게 된다 따라서 class의 역수가 -log안에 들어간다.* 



In [ ]:
# Complete the implementation of softmax_loss_naive and implement a (naive)
# version of the gradient that uses nested loops.
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 0.0)

# As we did for the SVM, use numeric gradient checking as a debugging tool.
# The numeric gradient should be close to the analytic gradient.
from cs231n.gradient_check import grad_check_sparse
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 0.0)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

# similar to SVM case, do another gradient check with regularization
loss, grad = softmax_loss_naive(W, X_dev, y_dev, 5e1)
f = lambda w: softmax_loss_naive(w, X_dev, y_dev, 5e1)[0]
grad_numerical = grad_check_sparse(f, W, grad, 10)

numerical: -0.915015 analytic: 0.149545, relative error: 1.000000e+00
numerical: 2.324238 analytic: 0.089270, relative error: 9.260244e-01
numerical: 1.797829 analytic: 0.063425, relative error: 9.318466e-01
numerical: 1.168867 analytic: 0.161932, relative error: 7.566391e-01
numerical: 0.851230 analytic: 0.072193, relative error: 8.436398e-01
numerical: 0.198005 analytic: 0.033168, relative error: 7.130448e-01
numerical: -1.777337 analytic: -0.014116, relative error: 9.842407e-01
numerical: -1.944642 analytic: 0.159579, relative error: 1.000000e+00
numerical: 4.267482 analytic: 0.205949, relative error: 9.079233e-01
numerical: 0.947831 analytic: 0.013578, relative error: 9.717535e-01
numerical: -1.738093 analytic: 0.024581, relative error: 1.000000e+00
numerical: 1.408786 analytic: 0.164616, relative error: 7.907511e-01
numerical: -1.000970 analytic: 0.063575, relative error: 1.000000e+00
numerical: 0.647163 analytic: 0.098187, relative error: 7.365346e-01
numerical: 0.942864 analytic

In [ ]:
# Now that we have a naive implementation of the softmax loss function and its gradient,
# implement a vectorized version in softmax_loss_vectorized.
# The two versions should compute the same results, but the vectorized version should be
# much faster.
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('naive loss: %e computed in %fs' % (loss_naive, toc - tic))

from cs231n.classifiers.softmax import softmax_loss_vectorized
tic = time.time()
loss_vectorized, grad_vectorized = softmax_loss_vectorized(W, X_dev, y_dev, 0.000005)
toc = time.time()
print('vectorized loss: %e computed in %fs' % (loss_vectorized, toc - tic))

# As we did for the SVM, we use the Frobenius norm to compare the two versions
# of the gradient.
grad_difference = np.linalg.norm(grad_naive - grad_vectorized, ord='fro')
print('Loss difference: %f' % np.abs(loss_naive - loss_vectorized))
print('Gradient difference: %f' % grad_difference)

naive loss: 2.326953e+00 computed in 0.139955s
vectorized loss: 2.326953e+00 computed in 0.007853s
Loss difference: 0.000000
Gradient difference: 21.195860


In [ ]:
# Use the validation set to tune hyperparameters (regularization strength and
# learning rate). You should experiment with different ranges for the learning
# rates and regularization strengths; if you are careful you should be able to
# get a classification accuracy of over 0.35 on the validation set.

from cs231n.classifiers import Softmax
results = {}
best_val = -1
best_softmax = None

################################################################################
# TODO:                                                                        #
# Use the validation set to set the learning rate and regularization strength. #
# This should be identical to the validation that you did for the SVM; save    #
# the best trained softmax classifer in best_softmax.                          #
################################################################################

# Provided as a reference. You may or may not want to change these hyperparameters
learning_rates = [1e-7, 5e-7]
regularization_strengths = [2.5e4, 5e4]

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    
# Print out results.
for lr, reg in sorted(results):
    train_accuracy, val_accuracy = results[(lr, reg)]
    print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
                lr, reg, train_accuracy, val_accuracy))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

In [ ]:
# evaluate on test set
# Evaluate the best softmax on test set
y_test_pred = best_softmax.predict(X_test)
test_accuracy = np.mean(y_test == y_test_pred)
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

**Inline Question 2** - *True or False*

Suppose the overall training loss is defined as the sum of the per-datapoint loss over all training examples. It is possible to add a new datapoint to a training set that would leave the SVM loss unchanged, but this is not the case with the Softmax classifier loss.

$\color{blue}{\textit Your Answer:True}$


$\color{blue}{\textit Your Explanation:
SVM은 일정 성능을 넘으면 개선하지 않는데 softmax는 확률이 1이 될 때까지 계속해서 성능을 개선한다.}$



In [ ]:
# Visualize the learned weights for each class
w = best_softmax.W[:-1,:] # strip out the bias
w = w.reshape(32, 32, 3, 10)

w_min, w_max = np.min(w), np.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(2, 5, i + 1)
    
    # Rescale the weights to be between 0 and 255
    wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
    plt.imshow(wimg.astype('uint8'))
    plt.axis('off')
    plt.title(classes[i])